# にほんごよみましょう

In [3]:
# --- Jupyter Notebook: Cell 1 (修正版) ---
#
# 必要なライブラリをインストールし、spaCy用の日本語モデルをダウンロードします。
print("必要なライブラリをインストールします...")
!pip install -U spacy
!pip install -U ginza

# 【重要】spaCyに'ja_ginza'モデルをダウンロードしてリンクするコマンド
print("\nspaCy用の日本語モデル 'ja_ginza' をダウンロードします...")
!python -m spacy download ja_ginza

print("\nインストールとモデルのダウンロードが完了しました。")

必要なライブラリをインストールします...
  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [79 lines of output]
      Ignoring numpy: markers 'python_version >= "3.9"' don't match your environment
        Using cached setuptools-75.3.2-py3-none-any.whl.metadata (6.9 kB)
        Using cached Cython-0.29.37-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata (3.1 kB)
        Using cached cymem-2.0.11-cp38-cp38-linux_x86_64.whl
        Using cached preshed-3.0.10.tar.gz (15 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyprojec

In [ ]:
# --- Jupyter Notebook: Cell 2 ---

import spacy
from collections import Counter
import sys

# GiNZAモデルをロードします（初回実行時やマシンのスペックによっては少し時間がかかる場合があります）
try:
    nlp = spacy.load('ja_ginza')
    print("spaCyとGiNZAモデルの読み込みが完了しました。")
except OSError:
    print("モデル 'ja_ginza' が見つかりません。")
    print("Cell 1でライブラリが正しくインストールされたか確認してください。")
    # Jupyter Notebook環境ではここで処理を中断するのが適切
    sys.exit()

In [ ]:
# --- Jupyter Notebook: Cell 4 ---

def logical_analysis(text: str):
    """
    与えられた日本語テキストに対して、論理構造の分析とキーワード抽出を行う関数。
    """
    # --- 辞書の定義 ---
    conjunctions = [
        "だから", "そのため", "したがって", "すると", "そこで", "ゆえに",
        "しかし", "だが", "でも", "けれど", "ところが", "とはいえ",
        "また", "そして", "それに", "かつ", "および",
        "または", "あるいは", "もしくは", "一方",
        "つまり", "すなわち", "なぜなら", "ただし", "ちなみに"
    ]
    five_w_one_h = ['いつ', 'どこ', '誰', '何', 'なぜ', 'どのよう']
    shijigo = [
        'これ', 'それ', 'あれ', 'この', 'その', 'あの',
        'ここ', 'そこ', 'あそこ', 'こちら', 'そちら', 'あちら',
        'こう', 'そう', 'ああ'
    ]
    buntmatsu_expressions = {
        "に違いない": "断定", "である": "断定", "たのである": "断定", "のである": "断定",
        "だろう": "推量", "でしょう": "推量",
        "かもしれない": "可能性",
        "ではないか": "疑問/提案", "だろうか": "疑問/提案"
    }
    
    # --- 1. GiNZAによるキーワード抽出 ---
    doc = nlp(text)
    nouns = [token.lemma_ for token in doc if token.pos_ == 'NOUN']
    keyword_counts = Counter(nouns)
    
    # --- 2. 各種マークアップ処理 ---
    marked_text = text
    
    # 接続詞、5W1H、指示語
    for conj in conjunctions:
        marked_text = marked_text.replace(conj, f"[接続詞:{conj}]")
    for word in five_w_one_h:
        marked_text = marked_text.replace(word, f"【5W1H:{word}】")
    for word in shijigo:
        marked_text = marked_text.replace(word, f"《指示語:{word}》")
        
    # 文末表現
    sentences = marked_text.split('。')
    processed_sentences = []
    sorted_expressions = sorted(buntmatsu_expressions.items(), key=lambda x: len(x[0]), reverse=True)
    
    for sentence in sentences:
        if not sentence: continue
        found = False
        for expression, category in sorted_expressions:
            if sentence.endswith(expression):
                cutoff = len(sentence) - len(expression)
                processed_sentence = sentence[:cutoff] + f"{{文末({category}):{expression}}}"
                processed_sentences.append(processed_sentence)
                found = True
                break
        if not found:
            processed_sentences.append(sentence)
    
    final_marked_text = "。".join(processed_sentences)
    if text.endswith('。'):
        final_marked_text += "。"
        
    return final_marked_text, keyword_counts

In [ ]:
# --- Jupyter Notebook: Cell 5 ---

# ファイル名の指定
input_filename = "input.txt"

try:
    with open(input_filename, "r", encoding="utf-8") as f:
        # ファイルからテキストを読み込む
        source_text = f.read()
        
        # 分析関数を実行
        marked_text, keywords = logical_analysis(source_text)
        
        # --- 結果の表示 ---
        print("--- 論理構造マークアップ テキスト ---")
        print(marked_text)
        print("\n" + "="*50 + "\n")
        print("--- キーワード出現回数ランキング ---")
        
        if keywords:
            for word, count in keywords.most_common(10): # 上位10件を表示
                print(f"- {word}: {count}回")
        else:
            print("キーワード（名詞）が見つかりませんでした。")

except FileNotFoundError:
    print(f"エラー: ファイル '{input_filename}' が見つかりません。")
    print("Jupyter Notebookと同じディレクトリにファイルを作成してください。")